# Numbers

[Numeric types](https://go.dev/ref/spec#Numeric_types) section in [The Go Programming Language Specification](https://go.dev/ref/)

Before we start, check Go version:

In [1]:
%%
fmt.Println(runtime. Version())

go1.23.4


## Integers

Go has 8 fixed-size integer types and two widely used aliases. From the [specs](https://go.dev/ref/spec#Numeric_types):

```go
uint8       // unsigned  8-bit integers (0 to 255)
uint16      // unsigned 16-bit integers (0 to 65535)
uint32      // unsigned 32-bit integers (0 to 4294967295)
uint64      // unsigned 64-bit integers (0 to 18446744073709551615)

int8        // signed  8-bit integers (-128 to 127)
int16       // signed 16-bit integers (-32768 to 32767)
int32       // signed 32-bit integers (-2147483648 to 2147483647)
int64       // signed 64-bit integers (-9223372036854775808 to 9223372036854775807)

float32     // IEEE 754 32-bit floating-point numbers
float64     // IEEE 754 64-bit floating-point numbers

complex64   // complex numbers with float32 real and imaginary parts
complex128  // complex numbers with float64 real and imaginary parts

byte        alias for uint8
rune        alias for int32
```

Go also has 3 integer types with implementation-dependent sizes:

```go
uint     // either 32 or 64 bits
int      // same size as uint
uintptr  // an unsigned integer large enough to store the uninterpreted bits of a pointer value
```

The integer types all wrap-around: if you add 1 to the maximum value, you get the minimum value, and vice-versa.

Examples:

In [2]:
%%
var u uint8
fmt.Println(u)
fmt.Println(u - 1)

0
255


In [3]:
%%
var u uint8 = 255
fmt.Println(u)
fmt.Println(u + 1)

255
0


In [4]:
%%
var i int8 = 127
fmt.Println(i + 1)

-128


In [5]:
%%
var i int8 = -128
fmt.Println(i - 1)

127


In [6]:
func pow2(exponent uint64) (result uint64) {
	result = 1
    var i uint64
	for i = 0; i < exponent; i++ {
		result *= 2
	}
	return result
}

In [7]:
%%
fmt.Println(pow2(63))
fmt.Println(pow2(64))
fmt.Println(pow2(64)-1)
const uint64max = 18446744073709551615  // https://go.dev/ref/spec#Numeric_types
fmt.Println(pow2(64)-1 == uint64max) 

9223372036854775808
0
18446744073709551615
true


### What's the size of my int?

In [8]:
%%
fmt.Println(bits.UintSize)

64


In [9]:
%%
uintSize := 32 << (^uint(0) >> 32 & 1) // 32 or 64 
fmt.Println(uintSize)

64


In [10]:
%%
fmt.Println(^uint32(0))

4294967295


In [11]:
%%
// for i := 0; i < 9; i++ {
for i := range 10 {
    j := ^uint8(0) >> i
    fmt.Printf("%d %3d %08b\n", i, j, j)
}


0 255 11111111
1 127 01111111
2  63 00111111
3  31 00011111
4  15 00001111
5   7 00000111
6   3 00000011
7   1 00000001
8   0 00000000
9   0 00000000


## The `byte8` and `rune` aliases

## Floats

In [12]:
%%
var f float64
for f = -20; f <= 20; f++ {
    bits := fmt.Sprintf("%064b", math.Float64bits(f))
    fmt.Printf("%5.1f  %v %v %v\n", f, bits[:1], bits[1:12], bits[12:])
}

-20.0  1 10000000011 0100000000000000000000000000000000000000000000000000
-19.0  1 10000000011 0011000000000000000000000000000000000000000000000000
-18.0  1 10000000011 0010000000000000000000000000000000000000000000000000
-17.0  1 10000000011 0001000000000000000000000000000000000000000000000000
-16.0  1 10000000011 0000000000000000000000000000000000000000000000000000
-15.0  1 10000000010 1110000000000000000000000000000000000000000000000000
-14.0  1 10000000010 1100000000000000000000000000000000000000000000000000
-13.0  1 10000000010 1010000000000000000000000000000000000000000000000000
-12.0  1 10000000010 1000000000000000000000000000000000000000000000000000
-11.0  1 10000000010 0110000000000000000000000000000000000000000000000000
-10.0  1 10000000010 0100000000000000000000000000000000000000000000000000
 -9.0  1 10000000010 0010000000000000000000000000000000000000000000000000
 -8.0  1 10000000010 0000000000000000000000000000000000000000000000000000
 -7.0  1 10000000001 11000000000000000

In [13]:
func parse(s string) int64 {
    i, err := strconv.ParseInt(s, 2, 64)
    if err != nil {
        panic(err)
    }
    return i
}

In [14]:
%%
var i int64
for i = range(11) {
    bits := fmt.Sprintf("%b", i)
    n := parse(bits)
    fmt.Printf("%4v %2d\n", bits, n)
}

   0  0
   1  1
  10  2
  11  3
 100  4
 101  5
 110  6
 111  7
1000  8
1001  9
1010 10


In [15]:
%%
var f float64
var sign rune
for f = -20; f <= 20; f++ {
    bits := math.Float64bits(f)
    bitStr := fmt.Sprintf("%064b", bits)
    if bits >> 63 == 1 {
        sign = '-'
    } else {
        sign = '+'
    }
    expStr := bitStr[1:12]
    exp := parse(expStr) - 1023
    fmt.Printf("%5.1f  %v %c  %v %5d %v\n", f, bitStr[:1], sign, expStr, exp, bitStr[12:])
}

-20.0  1 -  10000000011     4 0100000000000000000000000000000000000000000000000000
-19.0  1 -  10000000011     4 0011000000000000000000000000000000000000000000000000
-18.0  1 -  10000000011     4 0010000000000000000000000000000000000000000000000000
-17.0  1 -  10000000011     4 0001000000000000000000000000000000000000000000000000
-16.0  1 -  10000000011     4 0000000000000000000000000000000000000000000000000000
-15.0  1 -  10000000010     3 1110000000000000000000000000000000000000000000000000
-14.0  1 -  10000000010     3 1100000000000000000000000000000000000000000000000000
-13.0  1 -  10000000010     3 1010000000000000000000000000000000000000000000000000
-12.0  1 -  10000000010     3 1000000000000000000000000000000000000000000000000000
-11.0  1 -  10000000010     3 0110000000000000000000000000000000000000000000000000
-10.0  1 -  10000000010     3 0100000000000000000000000000000000000000000000000000
 -9.0  1 -  10000000010     3 0010000000000000000000000000000000000000000000000000
 -8.

In [16]:
%%
for i := range 64 {
    f := float64(i) / 8.0
    bits := math.Float64bits(f)
    fmt.Printf("%f  %064b\n", f, bits)
}

0.000000  0000000000000000000000000000000000000000000000000000000000000000
0.125000  0011111111000000000000000000000000000000000000000000000000000000
0.250000  0011111111010000000000000000000000000000000000000000000000000000
0.375000  0011111111011000000000000000000000000000000000000000000000000000
0.500000  0011111111100000000000000000000000000000000000000000000000000000
0.625000  0011111111100100000000000000000000000000000000000000000000000000
0.750000  0011111111101000000000000000000000000000000000000000000000000000
0.875000  0011111111101100000000000000000000000000000000000000000000000000
1.000000  0011111111110000000000000000000000000000000000000000000000000000
1.125000  0011111111110010000000000000000000000000000000000000000000000000
1.250000  0011111111110100000000000000000000000000000000000000000000000000
1.375000  0011111111110110000000000000000000000000000000000000000000000000
1.500000  0011111111111000000000000000000000000000000000000000000000000000
1.625000  001111111111101

In [17]:
%%
for i := range 10 {
    f := float64(i) / 10.0
    bits := math.Float64bits(f)
    fmt.Printf("%f  %064b\n", f, bits)
}

0.000000  0000000000000000000000000000000000000000000000000000000000000000
0.100000  0011111110111001100110011001100110011001100110011001100110011010
0.200000  0011111111001001100110011001100110011001100110011001100110011010
0.300000  0011111111010011001100110011001100110011001100110011001100110011
0.400000  0011111111011001100110011001100110011001100110011001100110011010
0.500000  0011111111100000000000000000000000000000000000000000000000000000
0.600000  0011111111100011001100110011001100110011001100110011001100110011
0.700000  0011111111100110011001100110011001100110011001100110011001100110
0.800000  0011111111101001100110011001100110011001100110011001100110011010
0.900000  0011111111101100110011001100110011001100110011001100110011001101


The set of `float64` is not a superset of `int64`.

In [18]:
%%
for e := range 64 {
    var i = 1 << e - 1
    var f = float64(i)
    var is = fmt.Sprintf("%d", i)
    var fs = fmt.Sprintf("%.f", f)
    if is != fs {
        fmt.Printf("%d\t%d\n\t%.1f\n", e, i, f)
    }
}


54	18014398509481983
	18014398509481984.0
55	36028797018963967
	36028797018963968.0
56	72057594037927935
	72057594037927936.0
57	144115188075855871
	144115188075855872.0
58	288230376151711743
	288230376151711744.0
59	576460752303423487
	576460752303423488.0
60	1152921504606846975
	1152921504606846976.0
61	2305843009213693951
	2305843009213693952.0
62	4611686018427387903
	4611686018427387904.0
63	9223372036854775807
	9223372036854775808.0


In [ ]:
%%
var start = 1 << 52 - 1
for i := start; i < start * 2; i++ {
    f := float64(i)
    var is = fmt.Sprintf("%d", i)
    var fs = fmt.Sprintf("%.f", f)
    if is != fs {
        fmt.Printf("%d\n%.1f\n", i, f)
        break
    }    
}

## Complex numbers

The types `complex64` and `complex128` represent complex numbers, represented as pairs of floats, as implemented in modern CPUs. 